In [1]:
import re
import json
import string
import pandas as pd
from cleantext import clean
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
reviews_raw = pd.read_csv('../outputs/reviews_raw.csv')[['group', 'review_raw']]
reviews_raw

,group,review_raw
0,cg_unique,"Alp-Us ist ein Serviceprovider, der das Planen..."
1,cg_unique,Alp-Us ist ein Tool für die Organisation eines...
2,cg_unique,Alp-Us ist eine innovative Anwendung zur Organ...
3,cg_unique,"Alp-us ist eine Anwendung, mit der Sie den ges..."
4,cg_unique,"Alp-us ist eine App, mit der Gruppen Skiausflü..."
...,...,...
174,gpt35_unique,Das vorgeschlagene Geschäftsmodell basiert auf...
175,gpt35_unique,"Die Geschäftsidee ""Alp-Us"" umfasst eine App, m..."
176,gpt35_unique,"Vielen Dank, dass Sie mir die Möglichkeit gege..."
177,gpt35_unique,"zuerst, Meiner Meinung nach ist es der beste D..."


In [3]:
gender_set = set()

with open('../scripts/male.txt', 'r') as file:
    for line in file:
        # Remove leading/trailing whitespace, including the newline character
        word = line.strip()
        gender_set.add(word) 

with open('../scripts/female.txt', 'r') as file:
    for line in file:
        # Remove leading/trailing whitespace, including the newline character
        word = line.strip()
        gender_set.add(word) 

In [4]:
def get_reviews(data_dir):
    with open(data_dir + 'reviews.json') as file:
        data = json.load(file)

    # Create a dictionary to store reviews by username
    reviews_by_username = {}
    cnt_none = 0
    # Process the reviews
    for review in data:
        username = review.get('username')
        review_text = review.get('review')
        review_time = review.get('time')

        # Skip reviews with less than 180 words
        if len(review_text.split()) < 180:
            print(username)
            continue

        if username is None:
            reviews_by_username['none' + str(cnt_none)] = {
                'review': review_text,
                'time': review_time
            }
            cnt_none += 1
        if username in reviews_by_username:
            existing_review = reviews_by_username[username]
            if review_time > existing_review['time']:
                existing_review['review'] = review_text
                existing_review['time'] = review_time
        else:
            reviews_by_username[username] = {
                'review': review_text,
                'time': review_time
            }

    # Get the unique reviews
    return reviews_by_username


In [5]:
def get_unique_reviews(reviews_by_username):
    return list(set([review['review'] for review in reviews_by_username.values()]))

In [ ]:
gpt2_reviews = get_reviews(review_dir + 'raw/gpt2/')
gpt3_reviews = get_reviews(review_dir + 'raw/gpt3/')
gpt35_reviews = get_reviews(review_dir + 'raw/gpt35/')
cg_reviews = get_reviews(review_dir + 'raw/cg/')

In [ ]:
gpt2_reviews

{'63020b564ab35295cccd386d': {'review': 'Ich finde das Geschäftsmodell grundsätzlich sehr gut. Ich würde die App auch benutzen um einen Ski Urlaub zu planen. Ich sehe viel potenzial in der Geschäftsidee.\n',
  'time': 1686056983},
 '608e74070357794c8c355e75': {'review': 'Alp-us schneit fur mich und meine Familie ad perfekte Tool zu sein, da wir eine Erfahrung in der Organisation von Reisen rund um den Wintersport haben. Besonders interessierte mich das Thema Hotellerie, Verkehr und Gastronomie. Ich denke, dass ich, eine Person mit wenig Erfahrung in der Organisation dieser Art von Reisen, sehr nutzlich sein kann. . somit garantierst du, ein sorgenfreies und gutes reisen, und präsentierst stärken meiner meinung nach, neben den bestehenden ideen, die youtube-revents. mir gefallt dieses tool sehr gut und ich mochte es zum beispiel in den kommenden ferien gerne nutzen. Ich habe keine besonderen negativen Kommentare\n',
  'time': 1686056942},
 '60fd425228a705aeec26c55c': {'review': "Der grö

In [ ]:
len(gpt2_reviews), len(gpt3_reviews), len(gpt35_reviews), len(cg_reviews)

(63, 58, 46, 46)

In [ ]:
# english review
cg_reviews['64539733186ad1764e4c2b74']

{'review': '<font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ALP US sounds as a comprehensive mobile application to help the ski enthusiast to plan their vacations. The video describes in details the aim of this application. The product is well described in the video, and tackles points such as how the user will be registered and on which aspects  it will help the user for example to plan their transport, accommodation, ski offering companies etc. In addition, a good point from the business model is the explanation of the link between the user, the product(application) and the service provider. To summarise, the product description in well explained in the video. However, In the business model explained in the video there is missing information, on the market analysis and marketing strategies, which market segment will be targeted and how will this be done.  As well as, the monetising strategy is not explained, how is this application(product) supposed to g

In [ ]:
# nonsense review
gpt35_reviews['5e3df819d96094073a9c939a']

{'review': 'Dieses geschaftsmodell ist sehr interessant.Es gibt ihnen die moglichkeit,uber die anwerdung orte zum skifahren zu buchen.Sie konnen unterkunft und gastronomie buchen.Es ware gut,transport,taxi,wascherei,zusatzliche attraktionen,ausfluge,usw.buchen sie konnen unterkunft und gastronomie buchen.es ware gut transport taxi wascherei zusatzliche attraktionen ausflugre buchen zu konnen spiele.veranstaltung sie konnen das angebot um die buchung von sportaurustsubg,skifahren overalls skiern schlittschuhen erweitern sie konnen auch unterkunfte im sommer in warmen landern am meer am meer buchen trinken saft essen eis einledung mit kono geheh theatr actor spielen machen rollen skie see net sehr net plotzlich net sehr gut Ich mag essen fahren ski laptop ferhnsehen shop shop shop schop schoping schop kino kino kino kino kino kino ,atraktionen spielrn spiele  spiele suppen suppen trinkrn getrenk=en essen essen schlafen schlafen reisen reisen reisen reisen reisen reisen reisen reisen soll

In [ ]:
del cg_reviews['64539733186ad1764e4c2b74'], gpt35_reviews['5e3df819d96094073a9c939a']
len(gpt2_reviews), len(gpt3_reviews), len(gpt35_reviews), len(cg_reviews)

(63, 58, 45, 45)

In [ ]:
gpt2_unique = get_unique_reviews(gpt2_reviews)
gpt3_unique = get_unique_reviews(gpt3_reviews)
gpt35_unique = get_unique_reviews(gpt35_reviews)
cg_unique = get_unique_reviews(cg_reviews)

extra35 = "Bezug auf das Alp-Us-Modells war die Gerschaftsstruktur gut aand gut eklart. Fur einen Beobachter liegen die Hauptvorteile der Nutzung der App in der Zentralisierung von Diensten und der möglichkeit, schneller und effektiver auf informationen zuzugreifen. außerdem können nutzer auf personalisierte inhalte zugreifen und ihre daten leicht verwalten. Die Vereinfachung des Kundenerlebnisses durch die Bewaltigung der wichigsten Herauforderungen einer Reise, wie Unterkunft, Transport und Verpflegung, ist ein Aspekt, nach dem die meisten Verbraucher suchen. Es ist klar, dass Alp-Us den logistischen Aspekt einer Reise erleichtert. Die plattform bietet eine benutzerfreundliche oberfläche, auf der flüge, unterkünfte und aktivitäten einfach gebucht und organisiert werden können. Die zusatzchile Funktion, mehrere Benutzer in einer Gruppe zuzullassen, ist interessant, es mussen jedoch weitere Details bereitgestellt werden, beispielsweise die Moglichkeit, mehrere Eingaben fur eine Reiseorganisation zu verknupfen. Es gibt viele Vorteile fur die Geschaftsseite, wenn eine Plattform wie diese Unternehmen wertvolle Daten bereitstellt. Vom Gesamtmodell her was es ansprechend. Der im Video gezeigt Hauptfehler war die Konzentraion auf die Idee, die Datumsauswahl fur einen Gruppenurlaub zu vereinfachen, obwohl es fur diese Art con Aufgabe hilfreich ist, die Verfugbarkeit von Skigebieten zu sehen. Der Hauptvorteil fur den Verbraucher besteht darin, dass er Zeit spart, indem er auf nur einer Plattform bucht, anstatt auf vier odder funf Plattform wie AirBnB, Tripadvisor und andere zuruckzugreifen. Darauf sollten sich zukunftige Marketingkampagnen kozentrieren. Ein weiterer Aspekt, den ich ansprechen mochte, ist die bedeutung von social media. In der heutigen digitalen welt ist eine starke Prasenz auf verschiedenen Social-Mediaa-Plattform unerlasslich. Dadurch konnen potenzielle Kunden auf das Produkt stoben und es nutzen."
gpt35_unique.append(extra35)

len(gpt2_unique), len(gpt3_unique), len(gpt35_unique), len(cg_unique)

(53, 47, 46, 40)

In [ ]:
# double check after remove duplication by python
cg_unique.sort()
cg_unique

['Alp-Us ist ein Serviceprovider, der das Planen von Ski-Urlauben vereinfachen Soll. Die App bildet eine Schnittstelle, an der Teilnehmer am Urlaub alle wichtigen Aspekte (Skigebiet, Wohnung, Transport, Gastronomie) verwalten können. Die Stärke an dem Angebot von Alp-Us ist die Einfachheit alle Bedienelemente an einem Ort (innerhalb 1er App) zu haben. Benutzer können innerhalb der App alle Teilnehmer organisierien und können die besten Angebote für Aspekte wie Unterkunft oder Gastronomie direkt über die Verlinkung zu anderen populären Apps (ie. Tripadvisor, Airbnb, Bahn). Der Fakt, dass die Angebote zu den verschiedenen Punkten von verschiedenen bekannten Anbietern kommt, vermittelt Teilnehmern das Gefühl sich aus den bestmöglichen Angeboten die für sie am besten passenden heraussuchen zu können. Partner wie beispielsweise Airbnb profitieren davon, dass Ihre Angebote direkt zielgerichtet an bereits interessierte Kunden vermittelt werden. Auch die Transparenz und Verfügbarkeit kundenspe

In [ ]:
cg_unique[8], cg_unique[7]

('Bei Alp-Us handelt es sich um eine besondere App zum Wintersporttourismus. Das Geschäftsmodell ist vielseitig und deckt alle wichtigen Bereiche für Skisportbegeisterte ab. Es ist sehr innovativ und hilfreich. Mit der App lassen sich alle Aspekte des Skiurlaubs zentral darüber planen. Egal, ob es sich um ein Familienausflug, eine Freundestour oder einen Betriebsausflug handelt, hier wird für alle das Passende generiert. Mit der App lässt sich passend für jeden Anlass ein gruppenspezifisch perfekter Ausflug für die Teilnehmer und Teilnehmerinnen gestalten. Es wird empfohlen, dass sich alle Gruppenmitglieder hier registrieren. Dann werden die persönlichen Präferenzen abgefragt, sowie die Mobilitätsoptionen der Teammitglieder. Anhand dieser persönlichen Daten und Vorgaben zeigt einem Alp-Us gut durchdachte Vorschläge zu den Bereichen Unterkünfte (zum Beispiel ist hier Airnnb vertreten), Transport, Gastronomie und Skigebiet. Die Teilnehmer können nun aus den ganzen Vorschlägen passende Op

In [ ]:
print(len(cg_unique))
del cg_unique[8]
len(cg_unique)

40


39

In [ ]:
cg_unique

['Alp-Us ist ein Serviceprovider, der das Planen von Ski-Urlauben vereinfachen Soll. Die App bildet eine Schnittstelle, an der Teilnehmer am Urlaub alle wichtigen Aspekte (Skigebiet, Wohnung, Transport, Gastronomie) verwalten können. Die Stärke an dem Angebot von Alp-Us ist die Einfachheit alle Bedienelemente an einem Ort (innerhalb 1er App) zu haben. Benutzer können innerhalb der App alle Teilnehmer organisierien und können die besten Angebote für Aspekte wie Unterkunft oder Gastronomie direkt über die Verlinkung zu anderen populären Apps (ie. Tripadvisor, Airbnb, Bahn). Der Fakt, dass die Angebote zu den verschiedenen Punkten von verschiedenen bekannten Anbietern kommt, vermittelt Teilnehmern das Gefühl sich aus den bestmöglichen Angeboten die für sie am besten passenden heraussuchen zu können. Partner wie beispielsweise Airbnb profitieren davon, dass Ihre Angebote direkt zielgerichtet an bereits interessierte Kunden vermittelt werden. Auch die Transparenz und Verfügbarkeit kundenspe

In [ ]:
# double check after remove duplication by python
gpt2_unique.sort()
gpt2_unique

['\nEs ist gut gemacht das zum start des videos erst einmal das problem dargestellt wird was dieses produkt lösen möchte. Damit wird die Zielgruppe direkt angesprochen\ndannach wird einem auf simpele art und weise erklärt wie man die App benutzt was die benutzerfreundlichkeit zeigt\ndarauf folgend sieht man das die App eine allround lösung für ski strips sind, von transport bis zu hotel und skianlage ist alles auf einem platz buchbar.\nzum ende der präsentation sieht man die nähere erklärung des geschäfftsmodels. An dieser erklärenden grafik kann man erkennen das dass geschäftsmodell der App im kerne Vermittlung ist. und dort beginnt auch die Kritik an dem Produkt.    \nDas Produkt bring keine grundlegende neue funktion das geschäftsmodell basiert auf andere anbieter was unsere verhandlungsmacht in der branche drastisch senkt.\nIn der Präsentation wird nicht auf den finanziellen Aspekt der Firma eingegangen. wie ist der cashflow? wie hoch sind die operating costs? Grundlegende fragen w

In [ ]:
gpt2_unique[9], gpt2_unique[10]

('AlpUs ist eine Plattform, die es ermöglicht, Skiurlaube mit Freunden und Familie einfacher zu organisieren. Dabei wird es ermöglicht den Urlaub selbst und ebenfalls die Anreise zu planen.\r\nHierbei hilft die Plattform dabei, die Urlaubszeiten abzustimmen und Präferenzen von den Mitfahrern abzufragen. \r\nEs werden automatisch Vorschläge zu Skiorten und anderen Freizeitaktivitäten, wie Restaurants oder Sehenswürdigkeiten gemacht. \nDie Plattform monetarisiert die getätigten Buchungen bei den Partnerunternehmen, wobei dem Endkunden keine zusätzlichen Kosten entstehen, sondern dieser nur die Vorteile der Plattform, wie leichte Planbarkeit, Übersichtlichkeit der Angebote, Vergleichen von Angeboten und anschließend die direkte Buchung hat. Damit verlagert man den Schwerpunkt des Wertaustausches vom Endkunden hin zum Plattformanbieter, was die Nachhaltigkeit der Plattform unterstreicht.\nEin großer Vorteil ist, wie leicht es dem Planer des Ausfluges gemacht wird, viele Wünsche zu vereinen

In [ ]:
gpt2_unique[13], gpt2_unique[14], gpt2_unique[15]

('Bei Schwächen war der Ton sehr leise und die Sprache war schnell. Es reizte mich nicht, weiter zuzuhören. Die Lösung schien zu Beginn schwach, da es den Anschein hatte, als würde sie nicht viel zu dem beitragen, was die Leute normalerweise tun. Das Bild am Anfang bewegte sich nicht, daher dachte ich, dass das Video entweder kaputt war oder das Lösungsteam zu faul war. Was das Konzept selbst angeht, mache ich mir Sorgen über die Reaktionsfähigkeit der Menschen, sich überhaupt an der Bewerbung zu beteiligen. Oft werden Fils geplant, weil die Leute sich nicht an die Vorschriften halten. Es könnte schwierig sein, jeden davon zu überzeugen, die Anwendung herunterzuladen und dann eine lange Liste zu beantworten. Was passiert außerdem, wenn die Listeneinstellungen wichtige Themen übersehen? Und was ist, wenn die angebotenen Veranstaltungsorte nicht ausreichen? Werden die verschiedenen Restaurants und Restaurants einen Aufpreis oder den gleichen Betrag oder einen Rabatt verlangen, wenn sie ü

In [ ]:
print(len(gpt2_unique))
del gpt2_unique[14], gpt2_unique[13], gpt2_unique[9], 
len(gpt2_unique)

53


50

In [ ]:
gpt2_unique

['\nEs ist gut gemacht das zum start des videos erst einmal das problem dargestellt wird was dieses produkt lösen möchte. Damit wird die Zielgruppe direkt angesprochen\ndannach wird einem auf simpele art und weise erklärt wie man die App benutzt was die benutzerfreundlichkeit zeigt\ndarauf folgend sieht man das die App eine allround lösung für ski strips sind, von transport bis zu hotel und skianlage ist alles auf einem platz buchbar.\nzum ende der präsentation sieht man die nähere erklärung des geschäfftsmodels. An dieser erklärenden grafik kann man erkennen das dass geschäftsmodell der App im kerne Vermittlung ist. und dort beginnt auch die Kritik an dem Produkt.    \nDas Produkt bring keine grundlegende neue funktion das geschäftsmodell basiert auf andere anbieter was unsere verhandlungsmacht in der branche drastisch senkt.\nIn der Präsentation wird nicht auf den finanziellen Aspekt der Firma eingegangen. wie ist der cashflow? wie hoch sind die operating costs? Grundlegende fragen w

In [ ]:
gpt3_unique.sort()
gpt3_unique

['<font style="vertical-align: inherit;"><font style="vertical-align: inherit;" class=""> I understand that as a young entrepreneur, creating <span id="replybuttons_image" style="font-< span="">funding for a innovative project can be very tasking. However,  have drafted a well detailed model and feedback below: Step 1: My first step will be to conducts a market research to better understand my target audience and their preferences, then i will develop a unique value propositions that will set my project ahead from other competitors. I order to make project a very unique , i will provide good value for my customers ad my value propositions\' would be very clear ad concise. The next step will e to determine my revenue streams like charging customers a fee for subscription ad advertising thereby having a estimate of my potential earnings. My next step will e to create a financial plan which will include  a start-up cost, projected revenue and expenses which i tur help me determine how muc

In [ ]:
gpt3_unique[0], gpt3_unique[8], gpt3_unique[9], gpt3_unique[13], gpt3_unique[14]

('<font style="vertical-align: inherit;"><font style="vertical-align: inherit;" class=""> I understand that as a young entrepreneur, creating <span id="replybuttons_image" style="font-< span="">funding for a innovative project can be very tasking. However,  have drafted a well detailed model and feedback below: Step 1: My first step will be to conducts a market research to better understand my target audience and their preferences, then i will develop a unique value propositions that will set my project ahead from other competitors. I order to make project a very unique , i will provide good value for my customers ad my value propositions\' would be very clear ad concise. The next step will e to determine my revenue streams like charging customers a fee for subscription ad advertising thereby having a estimate of my potential earnings. My next step will e to create a financial plan which will include  a start-up cost, projected revenue and expenses which i tur help me determine how muc

In [ ]:
print(len(gpt3_unique))
del gpt3_unique[14], gpt3_unique[9], gpt3_unique[0]
len(gpt3_unique)

47


44

In [ ]:
gpt3_unique

['<font style="vertical-align: inherit;"><font style="vertical-align: inherit;">iFur uns ist das (A) geschaftsmodell ein gut strukturierter rahmen, der sein wertversprechen und seine strategische ausrichtung effektiv kommuniziert. Das modell vermittelt ein klares verstandnis des zielmarkts und zeigt das skalierbarkeits und wachstmspotenzial des unternehmens auf. Es identifiziert mehrere einnahmequellen verringert die abhangigkeit von einer einzigen quelle und erholt die finanzstabilitat. Es gibt jedoch bereiche mit verbesserungspote zial. Starken: 1) Klares wertversprechen das geschaftsmodell kommuniziert effektiv das einzigartige wertversprechen von (A) model. Es hebt die wichtigsten vorteile und unterscheidunggsmer kmale hervor, die das unternehmen von seinen mitbewerbern unterscheiden. Einnahmequellen: Das geschaftsmodell zeigt einen diversifizierten ansatz zur generierung von einnahmen. Es identifiziert mehrere potenzielle einnahmequellen einschlieblich (A) model, was die abhangigk

In [ ]:
gpt35_unique.sort()
gpt35_unique

['"Alp-Us" ist ein Tool, das das Bergerlebnis angenehmer machen soll. Es gibt zahlreiche Funktionen, die es Benutzern, die diese Anwendung verwenden, ermoglichen, Ausfluge in die Berge zu organisieren, Wintersport zu betreiben und all dies zu tun, ohne in stressige oder demotivierende Situationen zu geraten. Aus unserer taglichen Erfahrung wissen wir, wie schwierig und stressig die Organisation einer Reise sein kann. "Alp-Us" hilft den Benutzern und erleichtert ihnen die Organisation. "Alp-Us" ist ein Tool, das fur Familiengruppen, Unternehmen und Freundeskreise gleichermaben nutzlich sein kann. Diese App verfugt uber ein Zentralisierungssystem, das es einem "Leiter" ermoglicht, interessierte Benutzer durch das Senden geeigneter Links zur Nutzung einzuladen. Dieses System vergindert, dass das Erlebnis zu ablenkend wird (was sicherlich die Qualitat des Dienstes verbessert).Um das Erlebnis noch authentischer zu gestalten, bietet "Alp-Us" auberdem die Moglichkeit, das Erlebnis bereits bei

In [ ]:
len(cg_unique), len(gpt2_unique), len(gpt3_unique), len(gpt35_unique)

(39, 50, 44, 46)

In [ ]:
# store cg_unique, gpt2_unique, gpt3_unique, gpt35_unique in a pandas dataframe
reviews_all = pd.DataFrame(columns=['group', 'review_raw', 'review_withstop', 'review_nostop', 'review_nostop_total'])
for reviews, group in [(cg_unique, 'cg_unique'), (gpt2_unique, 'gpt2_unique'), \
                    (gpt3_unique, 'gpt3_unique'), (gpt35_unique, 'gpt35_unique')]:
    for review in reviews:
        new_row = pd.DataFrame([{'group': group, 'review_raw': review}])
        reviews_all = pd.concat([reviews_all, new_row], ignore_index=True)

In [ ]:
reviews_all

,group,review_raw,review_withstop,review_nostop,review_nostop_total
0,cg_unique,"Alp-Us ist ein Serviceprovider, der das Planen...",NaN,NaN,NaN
1,cg_unique,Alp-Us ist ein Tool für die Organisation eines...,NaN,NaN,NaN
2,cg_unique,Alp-Us ist eine innovative Anwendung zur Organ...,NaN,NaN,NaN
3,cg_unique,"Alp-us ist eine Anwendung, mit der Sie den ges...",NaN,NaN,NaN
4,cg_unique,"Alp-us ist eine App, mit der Gruppen Skiausflü...",NaN,NaN,NaN
...,...,...,...,...,...
174,gpt35_unique,Das vorgeschlagene Geschäftsmodell basiert auf...,NaN,NaN,NaN
175,gpt35_unique,"Die Geschäftsidee ""Alp-Us"" umfasst eine App, m...",NaN,NaN,NaN
176,gpt35_unique,"Vielen Dank, dass Sie mir die Möglichkeit gege...",NaN,NaN,NaN
177,gpt35_unique,"zuerst, Meiner Meinung nach ist es der beste D...",NaN,NaN,NaN


### processing functions

In [6]:
def female_pronouns(sentences):
    results = []
    for sentence in sentences.split('.'):
        pattern = r"\b(\w+)\s(?:Sie|Ihr|Ihrem|Ihren|Ihrer|Ihres)\b"
        matches = re.findall(pattern, sentence)
        if matches:
            # print(sentence)
            sentence = re.sub(pattern, matches[0].split(' ')[0], sentence)
            # print(sentence)
        results.append(sentence)
    return '.'.join(results)

In [7]:
def clean_text(review):
    # remove html tags
    sentences = BeautifulSoup(review).get_text(" ")
    # replace indexing with a space
    sentences = re.sub(r"\d+[).]+", ' ', sentences)
    # remove .pdf files
    sentences = re.sub('\w*.pdf', '', sentences)
    # keep e3 values
    sentences = re.sub('[eE]3', 'E-three', sentences)
    # remove version numbers
    sentences = re.sub('[vV][12]', '', sentences)
    # replace new line with space
    sentences = sentences.replace('\n', ' ')
    # remove urls
    sentences = re.sub(r'http\S+', '', sentences)

    sentences = female_pronouns(sentences)
    # use package
    sentences = clean(sentences, no_emoji=True, lower=True,
        no_urls=True, no_emails=True, no_phone_numbers=True, no_numbers=True,
        no_digits=True, no_currency_symbols=True, no_punct=True,
        replace_with_url='', replace_with_email='', replace_with_phone_number='',
        replace_with_number='', replace_with_digit='', replace_with_currency_symbol='', lang='de')
    
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', sentences).strip()
    return text_no_doublespace

In [8]:
def stop_word_removal(x, remove_stop_words):
    token = x.split()
    german_stop_words = stopwords.words('german')
    stop_words = set(german_stop_words) - gender_set
    abbreviations = ['ggf.', 'ggf', 'vlt.', 'vlt', 'dh.', 'd.h.', 'd.h', 'd h', 'dh', \
            'zb.', 'z.b.', 'z.b', 'z b', 'zb', 'bsp.', 'bsp', 'bspw.', 'bspw', \
            'oä.', 'o.ä.', 'oä', 'ev.', 'ev', 'evtl.', 'evtl']
    if remove_stop_words == 1:
        cleaned = ' '.join([w for w in token if not w in list(stop_words) and not w in abbreviations])
    elif remove_stop_words == 2:
        cleaned = ' '.join([w for w in token if not w in set(german_stop_words)])
    else:
        cleaned = ' '.join([w for w in token if not w in abbreviations])
    return cleaned.replace('.', '\n')

In [9]:
def clean_full(review, remove_stop_words):
    # remove meaningless characters
    review = clean_text(review)
    review = stop_word_removal(review, remove_stop_words=remove_stop_words)
    # remove all characters before the first letter
    return review.lstrip(string.punctuation + string.whitespace)

In [11]:
reviews_raw['review_withstop'] = reviews_raw['review_raw'].apply(lambda x: clean_full(x, remove_stop_words=0))
reviews_raw['review_nostop'] = reviews_raw['review_raw'].apply(lambda x: clean_full(x, remove_stop_words=0))
reviews_raw['review_nostop_total'] = reviews_raw['review_raw'].apply(lambda x: clean_full(x, remove_stop_words=0))

In [14]:
reviews_raw.iloc[177]

group                                                       gpt35_unique
review_raw             zuerst, Meiner Meinung nach ist es der beste D...
review_withstop        zuerst meiner meinung nach ist es der beste di...
review_nostop          zuerst meiner meinung nach ist es der beste di...
review_nostop_total    zuerst meiner meinung nach ist es der beste di...
Name: 177, dtype: object

In [15]:
reviews_raw = reviews_raw.drop(177)

In [16]:
reviews_raw.to_csv('../outputs/reviews_raw.csv', index=True)
reviews_raw.to_excel('../outputs/reviews_raw.xlsx', index=True)

In [ ]:
len(cg_unique), len(gpt2_unique), len(gpt3_unique), len(gpt35_unique)

In [ ]:
withstop_cg = [clean_full(review, remove_stop_words=0) for review in cg_unique]
withstop_gpt2 = [clean_full(review, remove_stop_words=0) for review in gpt2_unique]
withstop_gpt3 = [clean_full(review, remove_stop_words=0) for review in gpt3_unique]
withstop_gpt35 = [clean_full(review, remove_stop_words=0) for review in gpt35_unique]

In [ ]:
# store cg_unique, gpt2_unique, gpt3_unique, gpt35_unique in a pandas dataframe
i = 0
for reviews in [withstop_cg, withstop_gpt2, withstop_gpt3, withstop_gpt35]:
    for review in reviews:
        reviews_all.loc[i, 'review_withstop'] = review
        i += 1
reviews_all

,group,review_raw,review_withstop,review_nostop,review_nostop_total
0,cg_unique,"Alp-Us ist ein Serviceprovider, der das Planen...",alpus ist ein serviceprovider der das planen v...,NaN,NaN
1,cg_unique,Alp-Us ist ein Tool für die Organisation eines...,alpus ist ein tool für die organisation eines ...,NaN,NaN
2,cg_unique,Alp-Us ist eine innovative Anwendung zur Organ...,alpus ist eine innovative anwendung zur organi...,NaN,NaN
3,cg_unique,"Alp-us ist eine Anwendung, mit der Sie den ges...",alpus ist eine anwendung mit der sie den gesam...,NaN,NaN
4,cg_unique,"Alp-us ist eine App, mit der Gruppen Skiausflü...",alpus ist eine app mit der gruppen skiausflüge...,NaN,NaN
...,...,...,...,...,...
174,gpt35_unique,Das vorgeschlagene Geschäftsmodell basiert auf...,das vorgeschlagene geschäftsmodell basiert auf...,NaN,NaN
175,gpt35_unique,"Die Geschäftsidee ""Alp-Us"" umfasst eine App, m...",die geschäftsidee alpus umfasst eine app mit d...,NaN,NaN
176,gpt35_unique,"Vielen Dank, dass Sie mir die Möglichkeit gege...",vielen dank dass sie mir die möglichkeit gegeb...,NaN,NaN
177,gpt35_unique,"zuerst, Meiner Meinung nach ist es der beste D...",zuerst meiner meinung nach ist es der beste di...,NaN,NaN


In [ ]:
nostop_cg = [clean_full(review, remove_stop_words=1) for review in cg_unique]
nostop_gpt2 = [clean_full(review, remove_stop_words=1) for review in gpt2_unique]
nostop_gpt3 = [clean_full(review, remove_stop_words=1) for review in gpt3_unique]
nostop_gpt35 = [clean_full(review, remove_stop_words=1) for review in gpt35_unique]

In [ ]:
# store cg_unique, gpt2_unique, gpt3_unique, gpt35_unique in a pandas dataframe
i = 0
for reviews in [nostop_cg, nostop_gpt2, nostop_gpt3, nostop_gpt35]:
    for review in reviews:
        reviews_all.loc[i, 'review_nostop'] = review
        i += 1
reviews_all

,group,review_raw,review_withstop,review_nostop,review_nostop_total
0,cg_unique,"Alp-Us ist ein Serviceprovider, der das Planen...",alpus ist ein serviceprovider der das planen v...,alpus serviceprovider der planen skiurlauben v...,NaN
1,cg_unique,Alp-Us ist ein Tool für die Organisation eines...,alpus ist ein tool für die organisation eines ...,alpus tool organisation skitrips freunden plan...,NaN
2,cg_unique,Alp-Us ist eine innovative Anwendung zur Organ...,alpus ist eine innovative anwendung zur organi...,alpus innovative anwendung organisation organi...,NaN
3,cg_unique,"Alp-us ist eine Anwendung, mit der Sie den ges...",alpus ist eine anwendung mit der sie den gesam...,alpus anwendung der sie gesamten reiseplanungs...,NaN
4,cg_unique,"Alp-us ist eine App, mit der Gruppen Skiausflü...",alpus ist eine app mit der gruppen skiausflüge...,alpus app der gruppen skiausflüge organisieren...,NaN
...,...,...,...,...,...
174,gpt35_unique,Das vorgeschlagene Geschäftsmodell basiert auf...,das vorgeschlagene geschäftsmodell basiert auf...,vorgeschlagene geschäftsmodell basiert zusamme...,NaN
175,gpt35_unique,"Die Geschäftsidee ""Alp-Us"" umfasst eine App, m...",die geschäftsidee alpus umfasst eine app mit d...,geschäftsidee alpus umfasst app der möglich ze...,NaN
176,gpt35_unique,"Vielen Dank, dass Sie mir die Möglichkeit gege...",vielen dank dass sie mir die möglichkeit gegeb...,vielen dank sie möglichkeit gegeben ihr geschä...,NaN
177,gpt35_unique,"zuerst, Meiner Meinung nach ist es der beste D...",zuerst meiner meinung nach ist es der beste di...,zuerst meinung der beste dienstleister je genu...,NaN


In [ ]:
nostop_total_cg = [clean_full(review, remove_stop_words=2) for review in cg_unique]
nostop_total_gpt2 = [clean_full(review, remove_stop_words=2) for review in gpt2_unique]
nostop_total_gpt3 = [clean_full(review, remove_stop_words=2) for review in gpt3_unique]
nostop_total_gpt35 = [clean_full(review, remove_stop_words=2) for review in gpt35_unique]

# store cg_unique, gpt2_unique, gpt3_unique, gpt35_unique in a pandas dataframe
i = 0
for reviews in [nostop_total_cg, nostop_total_gpt2, nostop_total_gpt3, nostop_total_gpt35]:
    for review in reviews:
        reviews_all.loc[i, 'review_nostop_total'] = review
        i += 1
reviews_all

,group,review_raw,review_withstop,review_nostop,review_nostop_total
0,cg_unique,"Alp-Us ist ein Serviceprovider, der das Planen...",alpus ist ein serviceprovider der das planen v...,alpus serviceprovider der planen skiurlauben v...,alpus serviceprovider planen skiurlauben verei...
1,cg_unique,Alp-Us ist ein Tool für die Organisation eines...,alpus ist ein tool für die organisation eines ...,alpus tool organisation skitrips freunden plan...,alpus tool organisation skitrips freunden plan...
2,cg_unique,Alp-Us ist eine innovative Anwendung zur Organ...,alpus ist eine innovative anwendung zur organi...,alpus innovative anwendung organisation organi...,alpus innovative anwendung organisation organi...
3,cg_unique,"Alp-us ist eine Anwendung, mit der Sie den ges...",alpus ist eine anwendung mit der sie den gesam...,alpus anwendung der sie gesamten reiseplanungs...,alpus anwendung gesamten reiseplanungsprozess ...
4,cg_unique,"Alp-us ist eine App, mit der Gruppen Skiausflü...",alpus ist eine app mit der gruppen skiausflüge...,alpus app der gruppen skiausflüge organisieren...,alpus app gruppen skiausflüge organisieren ver...
...,...,...,...,...,...
174,gpt35_unique,Das vorgeschlagene Geschäftsmodell basiert auf...,das vorgeschlagene geschäftsmodell basiert auf...,vorgeschlagene geschäftsmodell basiert zusamme...,vorgeschlagene geschäftsmodell basiert zusamme...
175,gpt35_unique,"Die Geschäftsidee ""Alp-Us"" umfasst eine App, m...",die geschäftsidee alpus umfasst eine app mit d...,geschäftsidee alpus umfasst app der möglich ze...,geschäftsidee alpus umfasst app möglich zentra...
176,gpt35_unique,"Vielen Dank, dass Sie mir die Möglichkeit gege...",vielen dank dass sie mir die möglichkeit gegeb...,vielen dank sie möglichkeit gegeben ihr geschä...,vielen dank möglichkeit gegeben geschäftsmodel...
177,gpt35_unique,"zuerst, Meiner Meinung nach ist es der beste D...",zuerst meiner meinung nach ist es der beste di...,zuerst meinung der beste dienstleister je genu...,zuerst meinung beste dienstleister je genutze ...


In [ ]:
reviews_all.to_csv('../outputs/reviews_all.csv', index=False)